# TASK 1 Pre-processing and cleaning

In [46]:
import pandas as pd
import json

In [3]:
company=pd.read_json('company_table.json')
company

,corp_id,companyName,category,categoryCode,license1StartDate
0,16444,SAGEBRUSH TRUCKING,General freight trucking long distance truck-load,484121,None
1,13112,GESTION 4001 INC,None,0,None
2,3317,HELLY ENVIRONMENTAL,Offices of Physicians Mental Health Specialists,621112,None
3,19095,CADRAIN FARMS INC,All Other Miscellaneous Crop Farming,111998,None
4,24007,DR RUFUS MESHACK,None,0,None
...,...,...,...,...,...
995,16291,KYLE GIBSON,Drinking places (alcoholic beverages),722410,None
996,1850,IAIN GIBSON PC,None,0,None
997,686,405706 BC LTD,None,0,None
998,10261,SOFTGEN INTL,None,0,None


In [5]:
company.to_json('company_table_new.json',orient="records")

In [6]:
address=pd.read_json('address_table.json')
address_transposed=address.transpose()
address_transposed.reset_index(drop=True, inplace=False)
address_transposed=address_transposed.drop('index',axis=1)
address_transposed



,corp_id,addressLine,addressCity,addressRegion,addressPostal,addressCountry
0,16444,64 Micmac Blvd W,Lethbridge,AB,T1K 5L7,CA
1,13112,4001 Blvd Crémazie E Bureau 100,Montréal,QC,H1Z 2L2,CA
2,3317,48 Amiens Cres SW,Calgary,AB,T2T 6E8,CA
3,19095,PO Box 122,Leask,SK,S0J 1M0,CA
4,24007,1 Paton St,St. John S,NL,A1B 4S8,CA
...,...,...,...,...,...,...
1837,19086,No Address Line Given,Denham Springs,LA,70726,US
1838,16217,1250 639 Fifth Ave S W,Calgary,AB,T2P 0M9,CA
1839,7093,120 Laconia Rd Suite 118,Tilton,NH,03276,US
1840,865,850 Sophia St Box 879,Prescott,ON,K0E 1T0,CA


In [8]:
address_transposed[address_transposed.isnull().any(axis=1)]

,corp_id,addressLine,addressCity,addressRegion,addressPostal,addressCountry
6,12955,1 Richview Rd Unit-A,Toronto,ON,None,CA
38,23879,212 Bergheim Rd,Rm of Aberdeen No. 373,SK,None,CA
46,13216,118 1755 Springfield Rd,Kelowna,BC,None,CA
84,9651,235 Un St NE,Salem,OR,None,US
149,14980,No Address Line Given,None,MB,None,CA
...,...,...,...,...,...,...
1671,10325,No Address Line Given,Watsonville,CA,None,US
1699,19019,19500 Ave Clark-Graham,Baie D'urfe,QC,None,CA
1767,15689,Palestras Gratuitas Março 2020 – a Importância...,None,None,None,None
1791,8816,5671525e-5bd8-4a80-8ab4-786d014983ef,San Rafael,CA,None,US


In [9]:
address_transposed.to_json('address_table_new.json',orient="records")

# Task 2 Data Pipelining

In [45]:
#%pip install pymysql
#%pip install elasticsearch-serverless

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

     -------------------------------------- 315.3/315.3 kB 3.2 MB/s eta 0:00:00


In [ ]:
import pymysql
from elasticsearch_serverless import Elasticsearch

In [8]:
mySQLuser = 'root'
mySQLpasswd = 'Belman@30'
mySQLhost = '127.0.0.1'
mySQLport = 3306
mySQLdatabase = 'company_address'

connection = pymysql.connect(user = mySQLuser, passwd = mySQLpasswd, host = mySQLhost, port = mySQLport, db = mySQLdatabase)

In [ ]:
# Schema creation and JSON insertion

In [19]:
#print(connection)
my_cursor = connection.cursor()

query_1='select * from company_address.company_table_new'
query_2='select * from company_address.address_table_new'

In [60]:
my_cursor.execute(query_1)
company_data = pd.read_sql(query_1, con=connection)


c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [66]:
company_data['license1StartDate'] = pd.to_datetime(company_data['license1StartDate'], format="%Y-%m-%d", errors='coerce')
company_data['license1StartDate'].fillna(pd.Timestamp('1900-01-01'), inplace=True)


In [61]:
my_cursor.execute(query_2)
address_data= pd.read_sql(query_2, con=connection)
address_data

c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,corp_id,addressLine,addressCity,addressRegion,addressPostal,addressCountry
0,16444,64 Micmac Blvd W,Lethbridge,AB,T1K 5L7,CA
1,13112,4001 Blvd Crémazie E Bureau 100,Montréal,QC,H1Z 2L2,CA
2,3317,48 Amiens Cres SW,Calgary,AB,T2T 6E8,CA
3,19095,PO Box 122,Leask,SK,S0J 1M0,CA
4,24007,1 Paton St,St. John S,NL,A1B 4S8,CA
...,...,...,...,...,...,...
1837,19086,No Address Line Given,Denham Springs,LA,70726,US
1838,16217,1250 639 Fifth Ave S W,Calgary,AB,T2P 0M9,CA
1839,7093,120 Laconia Rd Suite 118,Tilton,NH,03276,US
1840,865,850 Sophia St Box 879,Prescott,ON,K0E 1T0,CA


In [69]:
connection.close()

In [48]:


client = Elasticsearch(
  "https://e2004a85c7a24f8f91993ffa72615cee.es.us-west-2.aws.elastic.cloud:443",
  api_key="WG5ZeGZaRUJFU1V6NUJMdC1RVk06UGFQVVdyeU1SWEtzNzRoZGJTMmtSUQ=="
)

In [49]:
client.info()

# documents = [
#   {"index": {"_index": "company_address"}},
#   {"name": "foo", "title": "bar"},
# ]

# client.bulk(operations=documents)

ObjectApiResponse({'name': 'serverless', 'cluster_name': 'e2004a85c7a24f8f91993ffa72615cee', 'cluster_uuid': 'hWF8wVlSTySAm4EgwCKYXg', 'version': {'number': '8.11.0', 'build_flavor': 'serverless', 'build_type': 'docker', 'build_hash': '00000000', 'build_date': '2023-10-31', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '8.11.0', 'minimum_index_compatibility_version': '8.11.0'}, 'tagline': 'You Know, for Search'})

In [67]:
index_name1 = 'company'
if not client.indices.exists(index=index_name1):
    client.indices.create(index=index_name1)

In [62]:

index_name2 = 'address'
if not client.indices.exists(index=index_name2):
    client.indices.create(index=index_name2)

In [64]:
for i, row in address_data.iterrows():
    doc = row.to_dict()
    client.index(index=index_name2, body=doc)

In [68]:
for i, row in company_data.iterrows():
    doc = row.to_dict()
    client.index(index=index_name1, body=doc)

# Task 3 Synchronization changes code

In [ ]:
def sync_with_elasticsearch():
    my_cursor.execute("SELECT * FROM sync_log")
    logs = my_cursor.fetchall()

    for log in logs:
        table_name = log['table_name']
        operation_type = log['operation_type']
        record_id = log['record_id']
        
        my_cursor.execute(f"SELECT * FROM {table_name} WHERE id = %s", (record_id,))
        record = my_cursor.fetchone()

        es_index = f"{table_name.lower()}_index"

        if operation_type in['INSERT','UPDATE']:
            client.index(index=es_index, id=record_id, body=record)
        
        
        my_cursor.execute("DELETE FROM sync_log WHERE id = %s", (log['id'],))
        connection.commit()


sync_with_elasticsearch()


my_cursor.close()
connection.close()